In [1]:
import pandas as pd
import torch
import torchvision
import argparse
import os
import PIL

from torchvision import datasets, transforms

from main_dino import *
from custom_dataset import GerDataset

# Data errors

In [2]:
annot_df = {'img_name' : [],'classes' : [],'x' : [],'y' : [],'width' : [],'height' : []}
empty_list = []
for idx,i in enumerate(os.listdir('kaggle/train/class/')):
    ## Empty Labels
    try:
        tmp_df = pd.read_csv(f'kaggle/train/class/{i}',header=None)
        for j in tmp_df[0].values:
            annot_df['img_name'].append(f'kaggle/train/images/{i.split(".")[0]}.png')
            annot_df['classes'].append(j.split()[0])
            annot_df['x'].append(j.split()[1])
            annot_df['y'].append(j.split()[2])
            annot_df['width'].append(j.split()[3])
            annot_df['height'].append(j.split()[4])
    except pd.errors.EmptyDataError:
        empty_list.append(i)
        continue

## There is 7 images which did not labelled

In [3]:
empty_list

['07081.txt',
 '02528.txt',
 '04966.txt',
 '06229.txt',
 '04285.txt',
 '03798.txt',
 '02448.txt']

## Check custom torch dataset

In [4]:
data = GerDataset('kaggle/train')

## Train Model

In [5]:
!python main_dino.py

Using cache found in /home/vscode/.cache/torch/hub/facebookresearch_xcit_main
Will run the code on one GPU.
| distributed init (rank 0): env://
git:
  sha: bff71b0b997d89b392bfde937841212e4922c821, status: has uncommited changes, branch: main

arch: vit_base
batch_size_per_gpu: 64
clip_grad: 3.0
data_path: kaggle/train
dist_url: env://
drop_path_rate: 0.1
epochs: 100
freeze_last_layer: 1
global_crops_scale: (0.4, 1.0)
gpu: 0
local_crops_number: 8
local_crops_scale: (0.05, 0.4)
local_rank: 0
lr: 0.0005
min_lr: 1e-06
momentum_teacher: 0.996
norm_last_layer: True
num_workers: 10
optimizer: adamw
out_dim: 65536
output_dir: output/
patch_size: 16
rank: 0
saveckp_freq: 20
seed: 0
teacher_temp: 0.04
use_bn_in_head: False
use_fp16: True
warmup_epochs: 10
warmup_teacher_temp: 0.04
warmup_teacher_temp_epochs: 0
weight_decay: 0.04
weight_decay_end: 0.4
world_size: 1
/opt/conda/lib/python3.8/site-packages/torchvision/transforms/transforms.py:890: UserWarning: Argument interpolation should be of ty